In [13]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern


## Assignment 3


## Task 1 

 Use EI and the mean prediction to solve any single objective benchmark problem (e.g.
Ackley, Rosenblock, sphere etc.) with any single objective optimizer (preferably GA). 

Set max exact function evaluations to 50 (start with 50 design points). Was the solutions
found by EI better? (you can implement EI is you wish to)

In [37]:
def create_samples(dec_dim, samples, bounds):
    # create samples 
    x = lhs(dec_dim, samples)    
    # scale
    #lower = bounds[0]
    #upper = bounds[1]
    #x = x * (upper - lower) + lower    
    return np.asarray(x)

In [38]:
# Problem is the expensive function to evaluate.
# use ackley


# prob takes now all as vectors, need to change GA to do so too.
def problem(x):
    # if only one solution to calculate
    if x.shape[0] == 2:
        term1 = -20 * np.exp(-0.2 * np.sqrt(0.5 * (x[0]**2 + x[1]**2)))
        term2 = np.exp(0.5 * (np.cos(2 * np.pi * x[0]) + np.cos(2 * np.pi * x[1])))
    else:
        term1 = -20 * np.exp(-0.2 * np.sqrt(0.5 * (x[:,0]**2 + x[:,1]**2)))
        term2 = np.exp(0.5 * (np.cos(2 * np.pi * x[:,0]) + np.cos(2 * np.pi * x[:,1])))
    return term1 - term2 + np.exp(1) + 20


### Using mean pred

In [18]:
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor


def surrogate(model, x):
    return model.predict(x, return_std=True)

def posteori_mean_pred(x, model):
    return model.predict(x)
    
def opt_acq(x, y, model):
    x_samples = create_samples(2, 50, bounds)
    
    scores = posteori_mean_pred(x_, model)
    best_x = np.argmin(scores)
    
    sampled_scores = posteori_mean_pred(x_samples, model)
    best_sampled = np.argmin(sampled_scores)
    
    if x[best_x] < x_samples[best_sampled]:
        return best_x
    else:
        return best_sampled

mean_pred_result_all = []
times = 10
bounds = np.array([[-2, -2], [2, 2]])

for _ in range(times):
       
    x = create_samples(2, 50, bounds)
    y = problem(x)
    kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5)
    model = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y)
    
    fmax = 0           
    while fmax < 50:
        best_x = np.argmin(posteori_mean_pred(x, model))
        best_x = x[best_x]
        
        true_y = problem(best_x) # + 1 fmax..
        fmax += 1
    
        x = np.vstack((x, [best_x]))
        y = np.hstack((y, [true_y]))

        # update surr
        model.fit(x, y)
            
  
    print("==============================")
    fit = np.min(y)
    print("best y", fit)
    print("best x", x[np.argmin(y)])

    mean_pred_result_all.append(fit)

best y 1.1582425389399553
best x [0.17630633 0.01764189]
best y 2.410305440792584
best x [0.21093992 0.23536211]
best y 0.2524720165472729
best x [0.01499056 0.05623917]
best y 1.06739562972189
best x [0.16657781 0.00579117]
best y 0.3899724330096994
best x [0.01794127 0.07821266]
best y 0.4168110473219393
best x [0.03882171 0.07456865]
best y 0.9682534720941227
best x [0.10376224 0.11220651]
best y 0.5155728366532379
best x [0.06277627 0.07494959]
best y 0.5004957651371846
best x [0.03912415 0.08757061]
best y 0.28169109905709533
best x [0.03504639 0.05247884]


In [19]:
mean_pred_result_all

[1.1582425389399553,
 2.410305440792584,
 0.2524720165472729,
 1.06739562972189,
 0.3899724330096994,
 0.4168110473219393,
 0.9682534720941227,
 0.5155728366532379,
 0.5004957651371846,
 0.28169109905709533]

In [20]:
np.mean(mean_pred_result_all)

0.7961212279274982

### single opt GA

In [39]:
class real_GA:
    def __init__(self, problem, pop, pop_size, pm, bounds, di, order, fitness=None, max_func_evals=50, gen_max=50):
        self.problem = problem # problem function to solve
        self.pop = pop # pop array to hold binary population strings
        self.pm = pm # probability of mutation
        self.pop_size = pop_size # population size
        self.lbounds = bounds[0]
        self.ubounds = bounds[1]
        self.di = di # for crossover
        self.order = order # for mutation
        self.gen_max = gen_max # max generations
        self.max_func_evals = max_func_evals
        
        self.fitness = fitness # fitness array to hold calculated fitness values
        self.gen = 0 # current generation
        
    # start pop and evaluate each member in the pop
    def initialize(self):
        if len(self.pop) < 1:
            x1range = np.random.uniform(low=self.lbounds[0], high=self.ubounds[0], size=self.pop_size)
            x2range = np.random.uniform(low=self.lbounds[1], high=self.ubounds[1], size=self.pop_size)
            self.pop = np.stack((x1range, x2range), axis=-1)

        #self.pop_size = self.pop.shape[0]
        self.fitness = self.evaluate(x)


    # run the GA for one iteration
    def run(self):
        n = 0 # init iterations
        self.initialize()

        next_gen = [] # init next_gen population array
        for i in range(int(self.pop.shape[0]/2)):
            # select two individuals with deterministic tournament selection, append them in a next gen list for crossover
            i1, i2 = self.tour_select()
            s1 = self.pop[i1]
            s2 = self.pop[i2]
            next_gen.append(s1)
            next_gen.append(s2)
            
        # crossover. Happens every time
        next_gen = self.SBX(next_gen)
            
        # Mutation. happens if rand < pm for member in pop
        for i in range(self.pop_size):
            if np.random.rand() < self.pm:
                next_gen[i] = self.poly_mutation(next_gen[i], self.order)
         
        self.pop = np.asarray(next_gen) # add next gen to self pop               
        self.fitness = self.evaluate(x)
        
        n += 1
        self.gen += 1     
    
    # evaluate population members
    def evaluate(self, x):
        return problem(x)
        
        
    # deterministic binary tournament selection
    def tour_select(self):        
        cf = self.fitness
        b1 = np.argmin(cf) # get best member by fitness
        cf = np.delete(cf, b1) # remove it from cf
        b2 = np.argmin(cf) # get (2nd) best member by fitness
        return b1, b2 # return best and 2nd best members as parents
        
        
    # Simulated binary crossover (non-bounded)
    def SBX(self, parents):
        parents = np.asarray(parents)
        pop_size, num_var = parents.shape
        children = np.zeros_like(parents)
        for i in range(0, pop_size, num_var):
            p1 = (parents[i] + parents[i + 1]) / 2
            p2 = (parents[i] - parents[i + 1]) / 2
            beta = np.zeros(num_var)
            alpha = np.random.rand(num_var)
            bx = np.random.randint(0, high=2, size=num_var)
            beta[alpha <= 0.5] = (2 * alpha[alpha <= 0.5])**(1 / (self.di + 1))
            beta[alpha > 0.5] = (2 - 2 * alpha[alpha > 0.5])**(-1 / (self.di + 1))            
            beta = beta * ((-1)**bx)
            children[i] = p1 + beta * p2
            children[i + 1] = p1 - beta * p2
        return children
        
    
    # polynomial mutation for one pop member p
    def poly_mutation(self, p, order):
        children = np.array([0,0])
        for i in range(0, 2):
            pL = self.lbounds[i]
            pU = self.ubounds[i]
            u = np.random.random() # r [0,1]
            mp = 0
            dl = (2*u)**(1/1+order) - 1
            dr = 1 - (2*(1 - u))**(1/1+order) 
            if u <= 0.5:
                mp = p[i] + dl*(p[i] - pL)
            else:
                mp = p[i] + dr*(pU - p[i])
            children[i] = mp
        return children

## using EI

In [47]:
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor

EI_result_all = []

def surrogate(model, x):
    return model.predict(x, return_std=True)

from scipy.stats import norm
from scipy.special import ndtr

def EI(mean, std, max_val, tradeoff):
    imp = (mean - max_val - tradeoff)
    z = imp / std
    ei = imp * norm.cdf(z) + std * norm.pdf(z)
    ei[std == 0.0] = 0.0
    return ei
    

def expected_impr(x0, x_sample, y, model):
    mu, sigma = model.predict(x0, return_std=True)
    sigma = sigma.reshape(-1, 1)
    max_val = np.max(y)
    tradeoff = 0.01

    return EI(mu, sigma, max_val, tradeoff)


from scipy.optimize import minimize

def propose_location(X_sample, Y_sample, model, bounds):
    min_x = np.array([1.,1.]) # atleast return something
    
    def min_obj(X0):
        # Minimization objective is the negative acquisition function
        return -expected_impr(X0, X_sample, Y_sample, model)
    
    # Find the best optimum by starting from n_restart different random points.   
    
    # use GA here
    # params
    pop_s = Y_sample.shape[0]
    pm = 0.1
    bounds = np.array([[-2, -2], [2, 2]]) # variable bounds (lower, upper)
    gen_max = 50
    di = 2 # distribution index
    order = 20 # polynomial order param

    pop = create_samples(2, pop_s, bounds)
    ga = real_GA(min_obj, pop, pop_s, pm, bounds, di, order, fmax, gen_max)
    for _ in range(10):
        ga.run() # run ga for 20 iterations
    
    best_fit = np.argmin(ga.fitness)
    min_x = ga.pop[best_fit]      
            
    return min_x

EI_result_all = []
times = 10

for _ in range(times):
    x = create_samples(2, 50, bounds)
    y = problem(x)
    kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=1.5)
    
    model = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y)
    
    fmax = 0
    while fmax < 50:

        best_x = propose_location(x, y, model, bounds)
        #print(best_x)
        true_y = problem(best_x) # + 1 fmax..
        fmax += 1
        
        # add to data. have to keep x, y == pop.size to not break GA currently.
        worst_y = np.argmax(y)
        x = np.delete(x, worst_y, axis=0)
        y = np.delete(y, worst_y, axis=0)
        x = np.vstack((x, best_x))
        y = np.hstack((y, true_y))

        # update surr
        model.fit(x, y)
    
    print("==============================")
    fit = np.min(y)
    print("best y", fit)
    print("best x", x[np.argmin(y)])

    EI_result_all.append(fit)

best y 0.452634979296743
best x [0.08549487 0.02591514]
best y 0.9679218146447894
best x [0.02603835 0.15251075]
best y 0.02005199020325321
best x [-0.00541927  0.00388961]
best y 0.0
best x [0. 0.]
best y 0.6659817764888416
best x [0.11260981 0.0345579 ]
best y 0.161870191666857
best x [0.04094391 0.00579893]
best y 0.10194037976392067
best x [0.01274589 0.02544643]
best y 0.36237825552528236
best x [0.04895394 0.05800767]
best y 0.6883575322604791
best x [0.01905027 0.11931682]
best y 0.4393186513698133
best x [0.00313146 0.0874817 ]


In [48]:
EI_result_all

[0.452634979296743,
 0.9679218146447894,
 0.02005199020325321,
 0.0,
 0.6659817764888416,
 0.161870191666857,
 0.10194037976392067,
 0.36237825552528236,
 0.6883575322604791,
 0.4393186513698133]

In [49]:
np.mean(EI_result_all)

0.38604555712199795

## Results:

Both algorithms are operating with bounds of [-2, -2]. Both use Matern kernel as it seems to work ok with Ackley. Mean prediction samples 50 new points using LHS each iteration. Genetic algorithm uses non-bounded SBX which effects to the variation not sticking to the bounds. GA also runs 10 generations to optimize the acquisicition function. Increasing generations seemed to make GA miss more, probably due to the non bounded SBX. Also since my GA was simple and hardcoded I had to keep the population size at 50.

### Mean pred

Using mean prediction with 50 real function evaluations between 10 independent runs we gain mean of 0.796 for fitness value. Ackley test problem has global minimum at [0,0] with 0 value. As expected mean prediction seems to get stuck to local minimas around the 

### EI

Using EI and GA with 50 real function evaluations between 10 independet runs we gain mean of 0.38 for fitness value. It is almost twice better than with mean prediction and EI also has less worse results and most times atleast one independent run from 10 finds the global minimum of 0.

## Task 2

Solve any benchmark problems (K=2 and 5, n=10) with ParEGO and LCB.
Start with 109 design points. Compare the hypervolume of the solutions after 100 exact function
evaluations.

Trying Botorch to solve these. It leads to using UCB instead of LCB since Botorch assumes maximization.

https://botorch.org/tutorials/multi_objective_bo

In [23]:
import os
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

In [24]:
from botorch.test_functions import DTLZ2

problem = DTLZ2(dim=10, num_objectives=2)

In [25]:
problem.ref_point = torch.tensor([0.0,0.0])

In [26]:
problem.ref_point

tensor([0., 0.])

In [27]:
problem.bounds

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [34]:
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.sampling import draw_sobol_samples

NOISE_SE = torch.tensor([1., 1., 1., 1., 1.])

BATCH_SIZE = 4
NUM_RESTARTS = 5 if not SMOKE_TEST else 2
RAW_SAMPLES = 64 if not SMOKE_TEST else 4

standard_bounds = torch.zeros(5, problem.dim)
standard_bounds[1] = 1

def generate_initial_data(n=109):
    # generate training data
    train_x = draw_sobol_samples(bounds=problem.bounds,n=n, q=1).squeeze(1)
    train_obj_true = problem(train_x)
    train_obj = train_obj_true + torch.randn_like(train_obj_true) * NOISE_SE
    return train_x, train_obj, train_obj_true


def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    train_x = normalize(train_x, problem.bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i:i+1]
        train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
        models.append(
            FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [29]:
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.monte_carlo import qUpperConfidenceBound
from botorch.utils.sampling import sample_simplex

def optimize_qucb_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qucb acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, problem.bounds)).mean

    weights = sample_simplex(problem.num_objectives).squeeze()
    objective = GenericMCObjective(get_chebyshev_scalarization(weights=weights, Y=pred))
    acq_func = qUpperConfidenceBound(
        model=model,
        beta=0.5,
        sampler=sampler,
        objective=objective,
        #posteori_transform=,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 30},
        sequential=True,
    )
    # observe new values 
    new_x =  unnormalize(candidates.detach(), bounds=problem.bounds)
    new_obj_true = problem(new_x)
    
    new_obj = new_obj_true #+ torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true

In [30]:
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement


def optimize_qnparego_and_get_observation(model, train_x, train_obj, sampler):
    """Samples a set of random weights for each candidate in the batch, performs sequential greedy optimization 
    of the qNParEGO acquisition function, and returns a new candidate and observation."""
    train_x = normalize(train_x, problem.bounds)
    with torch.no_grad():
        pred = model.posterior(train_x).mean
    acq_func_list = []
    for _ in range(BATCH_SIZE):
        weights = sample_simplex(problem.num_objectives).squeeze()
        objective = GenericMCObjective(get_chebyshev_scalarization(weights=weights, Y=pred))
        acq_func = qNoisyExpectedImprovement(  # pyre-ignore: [28]
            model=model,
            objective=objective,
            X_baseline=train_x,
            sampler=sampler,
            prune_baseline=True,
        )
        acq_func_list.append(acq_func)
    # optimize
    candidates, _ = optimize_acqf_list(
        acq_function_list=acq_func_list,
        bounds=standard_bounds,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 30},
    )
    # observe new values 
    new_x =  unnormalize(candidates.detach(), bounds=problem.bounds)
    
    # Here happens function evaluation
    new_obj_true = problem(new_x)
    
    
    new_obj = new_obj_true # + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true

In [31]:
from botorch import fit_gpytorch_model
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.dominated import DominatedPartitioning

import time
import warnings


warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

N_BATCH = 100 if not SMOKE_TEST else 10
MC_SAMPLES = 64 if not SMOKE_TEST else 16

verbose = True

hvs_qparego, hvs_qucb, = [], []

n = 109

# call helper functions to generate initial training data and initialize model
train_x_qparego, train_obj_qparego, train_obj_true_qparego = generate_initial_data(n=109)
mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)

train_x_qucb, train_obj_qucb, train_obj_true_qucb = train_x_qparego, train_obj_qparego, train_obj_true_qparego

mll_qucb, model_qucb = initialize_model(train_x_qucb, train_obj_qucb)

# compute hypervolume
bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_true_qparego)
volume = bd.compute_hypervolume().item()

hvs_qparego.append(volume)
hvs_qucb.append(volume)


# run N_BATCH rounds of BayesOpt after the initial random batch
for iteration in range(1, N_BATCH + 1):    
    
    t0 = time.time()
    
    # fit the models
    fit_gpytorch_model(mll_qparego)
    fit_gpytorch_model(mll_qucb)

    
    # define the qEI and qNEI acquisition modules using a QMC sampler
    qparego_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
    qucb_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

    
    # optimize acquisition functions and get new observations
    new_x_qparego, new_obj_qparego, new_obj_true_qparego = optimize_qnparego_and_get_observation(
        model_qparego, train_x_qparego, train_obj_qparego, qparego_sampler
    )
    new_x_qucb, new_obj_qucb, new_obj_true_qucb = optimize_qucb_and_get_observation(
        model_qucb, train_x_qucb, train_obj_qucb, qucb_sampler
    )
            
    # update training points
    train_x_qparego = torch.cat([train_x_qparego, new_x_qparego])
    train_obj_qparego = torch.cat([train_obj_qparego, new_obj_qparego])
    train_obj_true_qparego = torch.cat([train_obj_true_qparego, new_obj_true_qparego])

    train_x_qucb = torch.cat([train_x_qucb, new_x_qucb])
    train_obj_qucb = torch.cat([train_obj_qucb, new_obj_qucb])
    train_obj_true_qucb = torch.cat([train_obj_true_qucb, new_obj_true_qucb])
    

    # update progress
    for hvs_list, train_obj in zip(
        (hvs_qparego, hvs_qucb), 
        ( train_obj_true_qparego, train_obj_true_qucb),
    ):
        # compute hypervolume
        bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj)
        volume = bd.compute_hypervolume().item()
        hvs_list.append(volume)

    # reinitialize the models so they are ready for fitting on next iteration
    # Note: we find improved performance from not warm starting the model hyperparameters
    # using the hyperparameters from the previous iteration
    mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)
    mll_qucb, model_qucb = initialize_model(train_x_qucb, train_obj_qucb)

    #print(train_x_qucb[0,0])
    
    t1 = time.time()
    
    if verbose:
        print(
            f"\nBatch {iteration:>2}: Hypervolume (qNParEGO, qUCB) = "
            f"({hvs_qparego[-1]:>4.2f}, {hvs_qucb[-1]:>4.2f}), "
            f"time = {t1-t0:>4.2f}.", end=""
        )
    else:
        print(".", end="")


print("pref_point", problem.ref_point)
print("==========")
print(train_obj_true_qucb[:5])
print("==========\n")
print(train_obj_true_qparego[:5])


Batch  1: Hypervolume (qNParEGO, qUCB) = (3.53, 3.53), time = 3.47.
Batch  2: Hypervolume (qNParEGO, qUCB) = (3.53, 3.53), time = 3.18.
Batch  3: Hypervolume (qNParEGO, qUCB) = (3.53, 3.53), time = 2.32.
Batch  4: Hypervolume (qNParEGO, qUCB) = (3.53, 3.62), time = 2.67.
Batch  5: Hypervolume (qNParEGO, qUCB) = (3.53, 3.62), time = 2.54.
Batch  6: Hypervolume (qNParEGO, qUCB) = (3.53, 3.62), time = 2.85.
Batch  7: Hypervolume (qNParEGO, qUCB) = (3.53, 3.62), time = 2.65.
Batch  8: Hypervolume (qNParEGO, qUCB) = (3.53, 3.63), time = 3.00.
Batch  9: Hypervolume (qNParEGO, qUCB) = (3.53, 3.63), time = 3.49.
Batch 10: Hypervolume (qNParEGO, qUCB) = (3.53, 3.63), time = 2.70.
Batch 11: Hypervolume (qNParEGO, qUCB) = (3.53, 3.63), time = 3.26.
Batch 12: Hypervolume (qNParEGO, qUCB) = (3.53, 3.63), time = 3.16.
Batch 13: Hypervolume (qNParEGO, qUCB) = (3.53, 3.66), time = 4.45.
Batch 14: Hypervolume (qNParEGO, qUCB) = (3.54, 3.66), time = 4.25.
Batch 15: Hypervolume (qNParEGO, qUCB) = (3.69,

### 5 objective DTLZ5

In [32]:
problem = DTLZ2(dim=10, num_objectives=5)
problem.ref_point = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0])

In [35]:
N_BATCH = 100 if not SMOKE_TEST else 10
MC_SAMPLES = 64 if not SMOKE_TEST else 16

verbose = True

hvs_qparego, hvs_qucb, = [], []

n = 109

# call helper functions to generate initial training data and initialize model
train_x_qparego, train_obj_qparego, train_obj_true_qparego = generate_initial_data(n=109)
mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)

train_x_qucb, train_obj_qucb, train_obj_true_qucb = train_x_qparego, train_obj_qparego, train_obj_true_qparego

mll_qucb, model_qucb = initialize_model(train_x_qucb, train_obj_qucb)

# compute hypervolume
bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_true_qparego)
volume = bd.compute_hypervolume().item()

hvs_qparego.append(volume)
hvs_qucb.append(volume)


# run N_BATCH rounds of BayesOpt after the initial random batch
for iteration in range(1, N_BATCH + 1):    
    
    t0 = time.time()
    
    # fit the models
    fit_gpytorch_model(mll_qparego)
    fit_gpytorch_model(mll_qucb)

    
    # define the qEI and qNEI acquisition modules using a QMC sampler
    qparego_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
    qucb_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)

    
    # optimize acquisition functions and get new observations
    new_x_qparego, new_obj_qparego, new_obj_true_qparego = optimize_qnparego_and_get_observation(
        model_qparego, train_x_qparego, train_obj_qparego, qparego_sampler
    )
    new_x_qucb, new_obj_qucb, new_obj_true_qucb = optimize_qucb_and_get_observation(
        model_qucb, train_x_qucb, train_obj_qucb, qucb_sampler
    )
            
    # update training points
    train_x_qparego = torch.cat([train_x_qparego, new_x_qparego])
    train_obj_qparego = torch.cat([train_obj_qparego, new_obj_qparego])
    train_obj_true_qparego = torch.cat([train_obj_true_qparego, new_obj_true_qparego])

    train_x_qucb = torch.cat([train_x_qucb, new_x_qucb])
    train_obj_qucb = torch.cat([train_obj_qucb, new_obj_qucb])
    train_obj_true_qucb = torch.cat([train_obj_true_qucb, new_obj_true_qucb])
    

    # update progress
    for hvs_list, train_obj in zip(
        (hvs_qparego, hvs_qucb), 
        ( train_obj_true_qparego, train_obj_true_qucb),
    ):
        # compute hypervolume
        bd = DominatedPartitioning(ref_point=problem.ref_point, Y=train_obj)
        volume = bd.compute_hypervolume().item()
        hvs_list.append(volume)

    # reinitialize the models so they are ready for fitting on next iteration
    # Note: we find improved performance from not warm starting the model hyperparameters
    # using the hyperparameters from the previous iteration
    mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)
    mll_qucb, model_qucb = initialize_model(train_x_qucb, train_obj_qucb)

    #print(train_x_qucb[0,0])
    
    t1 = time.time()
    
    if verbose:
        print(
            f"\nBatch {iteration:>2}: Hypervolume (qNParEGO, qUCB) = "
            f"({hvs_qparego[-1]:>4.2f}, {hvs_qucb[-1]:>4.2f}), "
            f"time = {t1-t0:>4.2f}.", end=""
        )
    else:
        print(".", end="")


print("pref_point", problem.ref_point)
print("==========")
print(train_obj_true_qucb[:5])
print("==========\n")
print(train_obj_true_qparego[:5])


Batch  1: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 6.90.
Batch  2: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 6.21.
Batch  3: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 5.59.
Batch  4: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 5.66.
Batch  5: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 6.27.
Batch  6: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 6.45.
Batch  7: Hypervolume (qNParEGO, qUCB) = (0.48, 0.47), time = 6.98.
Batch  8: Hypervolume (qNParEGO, qUCB) = (0.48, 0.48), time = 6.45.
Batch  9: Hypervolume (qNParEGO, qUCB) = (0.48, 0.48), time = 8.98.
Batch 10: Hypervolume (qNParEGO, qUCB) = (0.49, 0.49), time = 8.61.
Batch 11: Hypervolume (qNParEGO, qUCB) = (0.49, 0.50), time = 8.14.
Batch 12: Hypervolume (qNParEGO, qUCB) = (0.49, 0.50), time = 6.00.
Batch 13: Hypervolume (qNParEGO, qUCB) = (0.49, 0.52), time = 9.82.
Batch 14: Hypervolume (qNParEGO, qUCB) = (0.49, 0.52), time = 7.72.
Batch 15: Hypervolume (qNParEGO, qUCB) = (0.49,

## Results

Reference points for both ParEGO and UCB are [0.,0.] for 2 objective case and [0., 0., 0., 0., 0.] for 5 objective case.


Note that Botorch assumes maximization so thats why using UCB and reference point for HV at [0,0] works. Botorch uses parallel batch versions of the algorithms and calls each iteration the real objective function only once. Code will take long to run with 100 iterations, my pc gets about 9 seconds per iteration for 2 objective case and nearing to 30 seconds per iteration for the 5 objective case (with RTX 3070). Also iteration number goes up, generally each batch takes a bit longer to compute especially with the 5 objective case.
The given 109 starting points would suggest that bigger batch size than 4 could be used, but since that increases the time each iteration takes, I stuck with batch size of 4 even though possibly better results would be gotten using bigger batch size.


|  |    HV (qnParEGO) |  HV (UCB) |
| --- | --- | --- |
| DTLZ2 (K=2, n=10) | 7.00   | 7.32  
| DTLZ2 (K=5, n=10) | 0.61   | 1.22  


UCB seems to work better in both cases even though ParEGO is usually tiny bit better in the first batch or two.
